In [1]:
import random
import sys
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix
from scipy.spatial.distance import jaccard
from sklearn.metrics import jaccard_score

In [2]:
data = np.load('user_movie_rating.npy')

print("row example:",data[0])
print("shape:",data.shape)
print("number of unique movies:", len(np.unique(data[:,1])))
print("number of unique users:", len(np.unique(data[:,0])))

row example: [ 1 30  3]
shape: (65225506, 3)
number of unique movies: 17770
number of unique users: 103703


In [3]:
np.random.seed(0)

<br/><br/>
<font size="5">
Creating our sparse matrixes
</font>

In [4]:
rows = data[:,1] # movie ids
cols = data[:,0] # user ids
# we don't care about the actual rating, 
# we want to know if the movie was rated by the user or not
data_vals = np.full(data.shape[0],1)

# csr matrix with movies in rows and users in cols
# this is the main matrix representation we will use
csr_sparse = csr_matrix((data_vals, (rows,cols)))
csr_shape = csr_sparse.get_shape()

# we have one extra row and one extra col because our user and movie 
# indexing starts from 0, while our data starts from ID;s of 1.
# The first row and the first col can therefore be ignored
print("csr matrix shape:", csr_shape)
print("csr matrix non zero elements:", csr_sparse.count_nonzero())
print("non zero elements in row 1:", csr_sparse[0].count_nonzero())
print("non zero elements in col 1:", csr_sparse[:,0].count_nonzero())

csr matrix shape: (17771, 103704)
csr matrix non zero elements: 65225506
non zero elements in row 1: 0
non zero elements in col 1: 0


<br/><br/>
<font size="5">
Permutation matrix creation
</font>

In [43]:
# since we are allowed, instead of creating hash functions
# we will use permutations of our rows which are faster to calculate
n_permutations = 100

rows_unpermuted = range(csr_shape[0])
row_permutations = np.array([ np.random.permutation(rows_unpermuted)
                                    for i in range(n_permutations)]).T
print("permutations table shape:", row_permutations.shape)

permutations table shape: (17771, 100)


In [44]:
len(np.unique(row_permutations[:,0]))

17771

<br/><br/>
<font size="5">
Signature matrix creation
</font>
<br/>
<font size="2">
We will use a numpy array for this because we will fill all the values anyway
</font>

In [45]:
signature_matrix = np.full((csr_shape[1], n_permutations), np.inf)
print("signature matrix shape:",signature_matrix.shape)

csr_range_rows = range(csr_shape[0])
sign_matr_rows = range(signature_matrix.shape[0])

#indexing sparse matrix rows
for curr_csr_row in csr_range_rows:
    non_zero_col_indexes = csr_sparse[curr_csr_row].nonzero()[1]
    curr_permut_vals = row_permutations[curr_csr_row]

    #indexing signature matrix updatable params
    for curr_col in non_zero_col_indexes:
        signature_matrix[curr_col] = np.minimum(signature_matrix[curr_col],
                                                curr_permut_vals)

signature matrix shape: (103704, 100)


<br/><br/>
<font size="5">
Jaccard Similarity
</font>

In [46]:
n_bands = 16
sig_bands = [ math.floor(n_permutations/n_bands)*i for i in range(n_bands+1) ]
range_sig_bands = range(len(sig_bands)-1)
print(sig_bands)

[0, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96]


In [56]:
pairs = []
range_curr_band = range(csr_shape[1])
for curr_band_idx in range_sig_bands:
    
    curr_band = signature_matrix[:,sig_bands[curr_band_idx]:sig_bands[curr_band_idx+1]]
    

    curr_user_index = 0 #needs to reset for each band
    dict_of_hashes = {}
    for curr_usr_idx in range_curr_band:
        curr_hash = hash(tuple(curr_band[curr_usr_idx]))
        dict_of_hashes.setdefault(curr_hash, []).append(curr_user_index)
        curr_user_index += 1


    # add pairs to array from dict 
    for curr_key in dict_of_hashes:
        if len(dict_of_hashes[curr_key]) > 1:
            curr_list = dict_of_hashes[curr_key]
            range_curr_list = range(len(curr_list)-1)
            for i in range_curr_list:
                second_rng = range(i+1,len(curr_list))
                for j in second_rng:
                    new_pair = [curr_list[i], curr_list[j]]
                    pairs.append(new_pair)

In [57]:
unique_pairs, counts = np.unique(pairs, axis=0, return_counts=True)
unique_repeated = unique_pairs[counts > 1]
print(f"total pairs:{len(pairs)}, unique pairs:{len(unique_pairs)}")
print(f"repeated pairs: {len(unique_repeated)}")
print(f"{unique_repeated}")

total pairs:5507101, unique pairs:5486718
repeated pairs: 20219
[[     5  14459]
 [     5  39842]
 [     5  54265]
 ...
 [102809 103401]
 [103046 103483]
 [103320 103519]]


In [49]:
#unique_repeated = sorted(pairs, key=lambda x: x[0])

In [50]:
csr_sparse_t = csr_sparse.tocsc()

In [51]:
file_out = open("results.txt", "w")
file_out.close()
counter = 0
for i in unique_repeated:
     curr_score = 1 - jaccard(csr_sparse_t[:,i[0]].toarray(),csr_sparse_t[:,i[1]].toarray())
     if curr_score >= 0.5:
          counter += 1
          file_out = open("results.txt", "a")
          file_out.write(str(i[0])+","+" "+str(i[1])+"\n")
          file_out.close()
print(counter)

35
